In [1]:
import pandas as pd
import requests
import geocoder
import numpy as np
import json
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


#### Importing Dataset

In [2]:
df=pd.read_csv('oklahoma_medical_facilities_locations.csv')
df.head()

,HEALTH CARE FACILITY NAME,ADDRESS,CITY,STATE,ZIP,PHONE1,LONGITUDE,LATITUDE,LOCATION
0,21st Century Home Health Agency,1421 Northwest 150th Street,Edmond,OK,73013.0,(405) 348-3459,-97.537352,35.623744,"(35.623744, -97.537352)"
1,"3D Health Care, Inc.",116 West Main Street,Wilburton,OK,74578.0,(918) 465-9001,-95.310146,34.918616,"(34.918616, -95.310146)"
2,Aall Care Home Health,"4867 S. Sheridan Rd., Suite 701",Tulsa,OK,74145.0,(918) 622-6446,-95.904243,36.092264,"(36.092264, -95.904243)"
3,"ABC Medical Clinic, Inc.",1508 South Virginia,Atoka,OK,74525.0,(580) 889-3355,-96.140201,34.372799,"(34.372799, -96.140201)"
4,A Better Way of Care,"2828 NW 57th, Suite 315",Oklahoma City,OK,73112.0,(405) 819-4696,-97.563946,35.529659,"(35.529659, -97.563946)"


In [3]:
df.head()

,HEALTH CARE FACILITY NAME,ADDRESS,CITY,STATE,ZIP,PHONE1,LONGITUDE,LATITUDE,LOCATION
0,21st Century Home Health Agency,1421 Northwest 150th Street,Edmond,OK,73013.0,(405) 348-3459,-97.537352,35.623744,"(35.623744, -97.537352)"
1,"3D Health Care, Inc.",116 West Main Street,Wilburton,OK,74578.0,(918) 465-9001,-95.310146,34.918616,"(34.918616, -95.310146)"
2,Aall Care Home Health,"4867 S. Sheridan Rd., Suite 701",Tulsa,OK,74145.0,(918) 622-6446,-95.904243,36.092264,"(36.092264, -95.904243)"
3,"ABC Medical Clinic, Inc.",1508 South Virginia,Atoka,OK,74525.0,(580) 889-3355,-96.140201,34.372799,"(34.372799, -96.140201)"
4,A Better Way of Care,"2828 NW 57th, Suite 315",Oklahoma City,OK,73112.0,(405) 819-4696,-97.563946,35.529659,"(35.529659, -97.563946)"


In [4]:
df['ZIP']=df['ZIP'].fillna(0.0).astype(int)

In [5]:
df.drop(['STATE'],axis=1,inplace=True)
df.head()

,HEALTH CARE FACILITY NAME,ADDRESS,CITY,ZIP,PHONE1,LONGITUDE,LATITUDE,LOCATION
0,21st Century Home Health Agency,1421 Northwest 150th Street,Edmond,73013,(405) 348-3459,-97.537352,35.623744,"(35.623744, -97.537352)"
1,"3D Health Care, Inc.",116 West Main Street,Wilburton,74578,(918) 465-9001,-95.310146,34.918616,"(34.918616, -95.310146)"
2,Aall Care Home Health,"4867 S. Sheridan Rd., Suite 701",Tulsa,74145,(918) 622-6446,-95.904243,36.092264,"(36.092264, -95.904243)"
3,"ABC Medical Clinic, Inc.",1508 South Virginia,Atoka,74525,(580) 889-3355,-96.140201,34.372799,"(34.372799, -96.140201)"
4,A Better Way of Care,"2828 NW 57th, Suite 315",Oklahoma City,73112,(405) 819-4696,-97.563946,35.529659,"(35.529659, -97.563946)"


In [8]:
address = 'Oklahoma'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oklahoma are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oklahoma are 34.9550817, -97.2684063.


In [9]:
# create map of Oklahoma using latitude and longitude values
map_oklahoma = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city,adress in zip(df['LATITUDE'], df['LONGITUDE'], df['CITY'],df['ADDRESS']):
    label = '{}, {}'.format(city, adress)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_oklahoma)  
    
map_oklahoma

In [ ]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20181229' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Let's explore the neighborhoods in our dataframe.

In [13]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
oklahoma_city_data = df[df['CITY'] == 'Oklahoma City'].reset_index(drop=True)
oklahoma_city_data.head()

,HEALTH CARE FACILITY NAME,ADDRESS,CITY,ZIP,PHONE1,LONGITUDE,LATITUDE,LOCATION
0,A Better Way of Care,"2828 NW 57th, Suite 315",Oklahoma City,73112,(405) 819-4696,-97.563946,35.529659,"(35.529659, -97.563946)"
1,Abiding Home Health of Oklahoma City,5929 North May #204,Oklahoma City,73112,(405) 607-2302,-97.565848,35.531457,"(35.531457, -97.565848)"
2,Accentra Home Healthcare,"4350 Will Rogers Parkway, Suite 500",Oklahoma City,73108,(405) 488-2222,-97.597747,35.452096,"(35.452096, -97.597747)"
3,"Accentra Home Healthcare, Inc.","4350 Will Rogers Pkwy, Suite 500",Oklahoma City,73108,(405) 917-1094,-97.597747,35.452096,"(35.452096, -97.597747)"
4,"Accredo Health Group, Inc.","4901 West Reno, Suite 950",Oklahoma City,73127,(405) 942-3961,-97.607725,35.464552,"(35.464552, -97.607725)"


In [15]:
address = 'Oklahoma City'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oklahoma City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oklahoma City are 35.4729886, -97.5170536.


In [16]:
# create map of Oklahoma City using latitude and longitude values
map_oklahoma_city = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city,adress in zip(oklahoma_city_data['LATITUDE'], oklahoma_city_data['LONGITUDE'], oklahoma_city_data['CITY'],
                                 oklahoma_city_data['ADDRESS']):
    label = '{}, {}'.format(adress,city)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_oklahoma_city)  
    
map_oklahoma_city

In [18]:
oklahoma_city_data.drop_duplicates(subset='LONGITUDE',inplace=True)

In [19]:
oklahoma_city_venues = getNearbyVenues(names=oklahoma_city_data['ADDRESS'],
                                   latitudes=oklahoma_city_data['LATITUDE'],
                                   longitudes=oklahoma_city_data['LONGITUDE']
                                  )

2828 NW 57th, Suite 315
5929 North May #204
4350 Will Rogers Parkway, Suite 500
4901 West Reno, Suite 950
1501 Southeast 66th Street, Suite G
8524 South Western, Suite 105
2800 NW 36th Street, Suite 220
7416 N. Broadway, Suite D
3545 NW 58th Street, Suite 800
4045 NW 64th Street, #504
4801 N. Classen, Suite 237
1004 Southwest 104th Street
3500 NW 39th Street
9700 Mashburn Boulevard
7201 Northeast 107th Street
1101 North Dewey, Suite 3P16
1111 North Dewey Avenue
9828 Northeast 23rd Street, Suite 2
3838 NW 36th Street, Suite 108
3105 S. Meridian
6505 Northeast 50th Street
3301 Northwest 63rd Street
7417 North May
One N. W. 64th Street, Suite A
5401 North Portland, Suite 280
4005 N.W. Expressway, Suite 252
3909 South Western Avenue
3714 N. Portland
4300 South Shields Boulevard
6601 N. May, Suite B
10101 S. Pennsylvania, Suite B
1140 North Hudson
3100 Southwest 89th Street
618 Northwest 32nd Street
7100 South I-35 Service Road, Suite 7
6101 West Reno, Suite 800
200 Quatum Drive
3033 North 

In [20]:
oklahoma_city_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"2828 NW 57th, Suite 315",35.529659,-97.563946,Mediterranean Imports & Deli,35.528544,-97.565290,Mediterranean Restaurant
1,"2828 NW 57th, Suite 315",35.529659,-97.563946,S&B's Burger Joint,35.531394,-97.566118,Burger Joint
2,"2828 NW 57th, Suite 315",35.529659,-97.563946,Chelino's,35.531017,-97.565288,Mexican Restaurant
3,"2828 NW 57th, Suite 315",35.529659,-97.563946,Zorba's Mediterranean Cuisine,35.532412,-97.565230,Mediterranean Restaurant
4,"2828 NW 57th, Suite 315",35.529659,-97.563946,Cimarron Alliance Equality Center,35.529019,-97.566491,Non-Profit


In [21]:
print('There are {} uniques categories.'.format(len(oklahoma_city_venues['Venue Category'].unique())))

There are 213 uniques categories.


In [22]:
# one hot encoding
oklahoma_onehot = pd.get_dummies(oklahoma_city_venues[['Venue Category']], prefix="", prefix_sep="")

# add address column back to dataframe
oklahoma_onehot['ADDRESS'] = oklahoma_city_venues['Neighbourhood'] 

# move address column to the first column
fixed_columns = [oklahoma_onehot.columns[-1]] + list(oklahoma_onehot.columns[:-1])
oklahoma_onehot = oklahoma_onehot[fixed_columns]

oklahoma_onehot.head()

,ADDRESS,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Video Store,Vietnamese Restaurant,Waste Facility,Weight Loss Center,Whisky Bar,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio
0,"2828 NW 57th, Suite 315",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"2828 NW 57th, Suite 315",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"2828 NW 57th, Suite 315",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"2828 NW 57th, Suite 315",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"2828 NW 57th, Suite 315",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
oklahoma_grouped = oklahoma_onehot.groupby('ADDRESS').mean().reset_index()
oklahoma_grouped

,ADDRESS,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Video Store,Vietnamese Restaurant,Waste Facility,Weight Loss Center,Whisky Bar,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio
0,100 Southeast 59th Street,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,1000 North Lee Avenue,0.000000,0.000000,0.000000,0.0,0.038462,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,"1000 North Lincoln Boulevard, Suite 150",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,"1000 West Wilshire Blvd., Suite 123B",0.000000,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,1004 Southwest 104th Street,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.083333,0.000000,0.000000
5,"10101 S. Pennsylvania, Suite B",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
6,10301 Greenbriar Parkway,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.066667,0.0,0.000000,0.0,0.000000,0.000000,0.000000
7,"1044 Southwest 44th, Suite 350",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
8,105 Southeast 45th Street,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9,"1101 North Dewey, Suite 3P16",0.000000,0.000000,0.000000,0.0,0.035714,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000


#### Function to sort the venues in descending order

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

def return_most_common_venues_val(row, num_top_venues):
    row_categories_val = row.iloc[1:]
    row_categories_sorted_val = row_categories_val.sort_values(ascending=False)
    
    return row_categories_sorted_val.values[0:num_top_venues]

In [25]:
print(return_most_common_venues_val(oklahoma_grouped.iloc[3, :], 5))

[0.08333333333333333 0.08333333333333333 0.08333333333333333
 0.08333333333333333 0.08333333333333333]


### create the new dataframe and display the top 10 venues for each neighborhood

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ADDRESS']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['ADDRESS'] = oklahoma_grouped['ADDRESS']
#print(neighborhoods_venues_sorted.shape)
#print(oklahoma_grouped.shape)
#print(neighborhoods_venues_sorted.columns)
for ind in np.arange(oklahoma_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:11] = return_most_common_venues(oklahoma_grouped.iloc[ind, :], num_top_venues)

'''
neighborhoods_venues_sorted['ADDRESS'] = neighborhoods_venues_sorted['Neighborhood']
neighborhoods_venues_sorted.drop(columns='Neighborhood',inplace=True)

'''
# create a new dataframe
neighborhoods_venues_sorted_val = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_val['ADDRESS'] = oklahoma_grouped['ADDRESS']
#print(neighborhoods_venues_sorted_val.shape)

for ind in np.arange(oklahoma_grouped.shape[0]):
    neighborhoods_venues_sorted_val.iloc[ind, 1:11] = return_most_common_venues_val(oklahoma_grouped.iloc[ind, :], num_top_venues)




In [27]:
neighborhoods_venues_sorted_val.head()

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,100 Southeast 59th Street,0.333333,0.333333,0.166667,0.166667,0,0,0,0,0,0
1,1000 North Lee Avenue,0.0769231,0.0769231,0.0769231,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615
2,"1000 North Lincoln Boulevard, Suite 150",0.4,0.2,0.2,0.2,0,0,0,0,0,0
3,"1000 West Wilshire Blvd., Suite 123B",0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333
4,1004 Southwest 104th Street,0.166667,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333


In [28]:
neighborhoods_venues_sorted.head(10)

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,100 Southeast 59th Street,Discount Store,Fast Food Restaurant,American Restaurant,Grocery Store,Yoga Studio,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
1,1000 North Lee Avenue,Bar,Coffee Shop,Gym,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Shopping Mall,Burger Joint,Pub,Cocktail Bar
2,"1000 North Lincoln Boulevard, Suite 150",Pharmacy,Gym,Medical Center,Bookstore,Home Service,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop
3,"1000 West Wilshire Blvd., Suite 123B",Boutique,Asian Restaurant,Deli / Bodega,Cosmetics Shop,Café,Flower Shop,Gourmet Shop,Bar,Toy / Game Store,Performing Arts Venue
4,1004 Southwest 104th Street,Pizza Place,Ice Cream Shop,Jewelry Store,Gas Station,Grocery Store,Tea Room,Kids Store,Athletics & Sports,Snack Place,Deli / Bodega
5,"10101 S. Pennsylvania, Suite B",Pizza Place,Insurance Office,Construction & Landscaping,Gas Station,Mexican Restaurant,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food
6,10301 Greenbriar Parkway,Mexican Restaurant,Bakery,Cosmetics Shop,Pet Store,Shipping Store,Weight Loss Center,Construction & Landscaping,Insurance Office,Pizza Place,Chinese Restaurant
7,"1044 Southwest 44th, Suite 350",Pharmacy,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
8,105 Southeast 45th Street,Pharmacy,Discount Store,Gas Station,Mobile Phone Shop,Fast Food Restaurant,Bowling Alley,Design Studio,Deli / Bodega,Food Court,Food & Drink Shop
9,"1101 North Dewey, Suite 3P16",Coffee Shop,Bar,Gym,Market,Latin American Restaurant,Street Food Gathering,German Restaurant,Bowling Alley,Breakfast Spot,Shopping Mall


### Using K means clustering to generate neighborhood clusters

In [29]:
kclusters = 7

oklahoma_grouped_clustering = neighborhoods_venues_sorted_val.iloc[:,[1,2,3,4]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(oklahoma_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 1, 0, 1, 1, 3, 1, 3, 6, 1, 1, 1, 3, 1, 1, 4, 3, 1, 2, 6, 5, 3,
       1, 3, 1, 4, 0, 6, 1, 6, 1, 6, 6, 2, 1, 1, 1, 0, 0, 1, 6, 2, 1, 1,
       1, 1, 2, 3, 6, 1, 2, 6, 4, 4, 6, 2, 1, 1, 3, 3, 5, 1, 6, 6, 6, 6,
       3, 1, 6, 1, 1, 3, 1, 1, 1, 6, 6, 3, 1, 1, 1, 1, 6, 0, 6, 1, 2, 3,
       6, 5, 6, 1, 3, 6, 1, 6, 6, 1, 1, 6, 3, 1, 6, 1, 1, 6, 1, 6, 1, 3,
       1, 1, 0, 6, 1, 1, 6, 6, 3, 1, 1, 5, 5, 0, 1, 3, 5, 3, 1, 6, 3, 1,
       6, 3, 1, 3, 6, 5, 1, 3, 5, 1, 3, 3, 6, 5, 6])

In [30]:
neighborhoods_venues_sorted_val['Cluster Labels'] = kmeans.labels_

In [31]:
neighborhoods_venues_sorted_val.head()

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,100 Southeast 59th Street,0.333333,0.333333,0.166667,0.166667,0,0,0,0,0,0,3
1,1000 North Lee Avenue,0.0769231,0.0769231,0.0769231,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615,0.0384615,1
2,"1000 North Lincoln Boulevard, Suite 150",0.4,0.2,0.2,0.2,0,0,0,0,0,0,0
3,"1000 West Wilshire Blvd., Suite 123B",0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,1
4,1004 Southwest 104th Street,0.166667,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,1


In [32]:
neighborhoods_venues_sorted

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,100 Southeast 59th Street,Discount Store,Fast Food Restaurant,American Restaurant,Grocery Store,Yoga Studio,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
1,1000 North Lee Avenue,Bar,Coffee Shop,Gym,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Shopping Mall,Burger Joint,Pub,Cocktail Bar
2,"1000 North Lincoln Boulevard, Suite 150",Pharmacy,Gym,Medical Center,Bookstore,Home Service,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop
3,"1000 West Wilshire Blvd., Suite 123B",Boutique,Asian Restaurant,Deli / Bodega,Cosmetics Shop,Café,Flower Shop,Gourmet Shop,Bar,Toy / Game Store,Performing Arts Venue
4,1004 Southwest 104th Street,Pizza Place,Ice Cream Shop,Jewelry Store,Gas Station,Grocery Store,Tea Room,Kids Store,Athletics & Sports,Snack Place,Deli / Bodega
5,"10101 S. Pennsylvania, Suite B",Pizza Place,Insurance Office,Construction & Landscaping,Gas Station,Mexican Restaurant,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food
6,10301 Greenbriar Parkway,Mexican Restaurant,Bakery,Cosmetics Shop,Pet Store,Shipping Store,Weight Loss Center,Construction & Landscaping,Insurance Office,Pizza Place,Chinese Restaurant
7,"1044 Southwest 44th, Suite 350",Pharmacy,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
8,105 Southeast 45th Street,Pharmacy,Discount Store,Gas Station,Mobile Phone Shop,Fast Food Restaurant,Bowling Alley,Design Studio,Deli / Bodega,Food Court,Food & Drink Shop
9,"1101 North Dewey, Suite 3P16",Coffee Shop,Bar,Gym,Market,Latin American Restaurant,Street Food Gathering,German Restaurant,Bowling Alley,Breakfast Spot,Shopping Mall


In [33]:
oklahoma_merged = oklahoma_city_data

In [34]:
oklahoma_merged = oklahoma_merged.join(neighborhoods_venues_sorted_val.set_index('ADDRESS'), on='ADDRESS')

In [35]:
oklahoma_merged.sort_values('ADDRESS').shape

(151, 19)

In [36]:
oklahoma_merged = oklahoma_merged.join(neighborhoods_venues_sorted.set_index('ADDRESS'), on='ADDRESS',rsuffix='_name')

In [37]:
oklahoma_merged.dropna(subset=['Cluster Labels'], inplace=True)

In [38]:
oklahoma_merged.shape

(147, 29)

In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, cname in zip(oklahoma_merged['LATITUDE'], oklahoma_merged['LONGITUDE'], oklahoma_merged['ADDRESS'], oklahoma_merged['Cluster Labels'], oklahoma_merged['1st Most Common Venue_name']):
    label = folium.Popup(str(poi) + '<br>Best place nearby:' + str(cname), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=('<b>'+str(poi)+'</b>' + '<br>Best place nearby:' + str(cname)),
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

### Let's Examine each cluster

In [40]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 0, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
5,"1501 Southeast 66th Street, Suite G",-97.486494,35.399600,"(35.3996, -97.486494)",0.375,0.125,0.125,0.125,0.125,0.125,...,Lingerie Store,Pizza Place,Cosmetics Shop,Home Service,Food,Clothing Store,Electronics Store,Food Court,Food & Drink Shop,Flower Shop
48,701 Southeast 82nd Street,-97.499383,35.384705,"(35.384705, -97.499383)",0.4,0.2,0.2,0.2,0,0,...,Hotel,Convenience Store,Karaoke Bar,Gun Shop,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop
63,"6400 North Santa Fe, Suite B",-97.511940,35.536958,"(35.536958, -97.51194)",0.333333,0.166667,0.166667,0.166667,0.166667,0,...,Flower Shop,Gym / Fitness Center,Hotel,Medical Center,Gas Station,Electronics Store,Food Court,Food & Drink Shop,Food,Flea Market
73,"2301 Northwest 39th Street, Suite 222",-97.552147,35.511429,"(35.511429, -97.552147)",0.5,0.1,0.1,0.1,0.1,0.1,...,Gay Bar,Hotel,German Restaurant,Martial Arts Dojo,Café,Print Shop,Department Store,Fabric Shop,Food & Drink Shop,Food
74,"2301 W. I44 Service Rd., Suite 101",-97.552029,35.512670,"(35.51267, -97.552029)",0.454545,0.0909091,0.0909091,0.0909091,0.0909091,0.0909091,...,Gay Bar,Ice Cream Shop,Café,Martial Arts Dojo,Park,Print Shop,Hotel,Arcade,Fabric Shop,Fried Chicken Joint
108,"1000 North Lincoln Boulevard, Suite 150",-97.502299,35.477074,"(35.477074, -97.502299)",0.4,0.2,0.2,0.2,0,0,...,Pharmacy,Gym,Medical Center,Bookstore,Home Service,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop
158,4400 North Lincoln Boulevard,-97.502927,35.515969,"(35.515969, -97.502927)",0.333333,0.166667,0.166667,0.166667,0.166667,0,...,Pizza Place,Wings Joint,Construction & Landscaping,Food,Museum,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Flower Shop


In [41]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 1, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
0,"2828 NW 57th, Suite 315",-97.563946,35.529659,"(35.529659, -97.563946)",0.117647,0.117647,0.0588235,0.0588235,0.0588235,0.0588235,...,Mediterranean Restaurant,Sandwich Place,Music Store,Bookstore,Mexican Restaurant,Sports Bar,Burger Joint,Fast Food Restaurant,Salon / Barbershop,Restaurant
1,5929 North May #204,-97.565848,35.531457,"(35.531457, -97.565848)",0.129032,0.0645161,0.0645161,0.0645161,0.0645161,0.0322581,...,Fast Food Restaurant,Pet Store,Sandwich Place,American Restaurant,Mediterranean Restaurant,Nightclub,Shoe Store,Burger Joint,Salon / Barbershop,Restaurant
7,"2800 NW 36th Street, Suite 220",-97.564502,35.507506,"(35.507506, -97.564502)",0.0714286,0.0714286,0.0714286,0.0714286,0.0714286,0.0714286,...,Thai Restaurant,Sandwich Place,Café,Burger Joint,Donut Shop,Gay Bar,Fast Food Restaurant,Turkish Restaurant,Racetrack,Discount Store
9,"3545 NW 58th Street, Suite 800",-97.580340,35.530029,"(35.530029, -97.58034)",0.166667,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,...,Mexican Restaurant,Restaurant,Supermarket,Pharmacy,Breakfast Spot,Furniture / Home Store,Bar,Coffee Shop,Bakery,Gift Shop
10,"4045 NW 64th Street, #504",-97.591726,35.538750,"(35.53875, -97.591726)",0.0909091,0.0909091,0.0909091,0.0909091,0.0909091,0.0909091,...,Music Store,Fast Food Restaurant,Wings Joint,Mediterranean Restaurant,Deli / Bodega,American Restaurant,Gym,Sporting Goods Shop,Seafood Restaurant,Steakhouse
12,1004 Southwest 104th Street,-97.532371,35.363150,"(35.36315, -97.532371)",0.166667,0.0833333,0.0833333,0.0833333,0.0833333,0.0833333,...,Pizza Place,Ice Cream Shop,Jewelry Store,Gas Station,Grocery Store,Tea Room,Kids Store,Athletics & Sports,Snack Place,Deli / Bodega
13,3500 NW 39th Street,-97.580792,35.511175,"(35.511175, -97.580792)",0.1,0.1,0.1,0.1,0.1,0.1,...,Breakfast Spot,Intersection,Fast Food Restaurant,Supermarket,Bar,Video Store,Indian Restaurant,Massage Studio,Wings Joint,Grocery Store
17,"1101 North Dewey, Suite 3P16",-97.522791,35.478886,"(35.478886, -97.522791)",0.107143,0.0714286,0.0714286,0.0357143,0.0357143,0.0357143,...,Coffee Shop,Bar,Gym,Market,Latin American Restaurant,Street Food Gathering,German Restaurant,Bowling Alley,Breakfast Spot,Shopping Mall
18,1111 North Dewey Avenue,-97.522822,35.478986,"(35.478986, -97.522822)",0.115385,0.0769231,0.0384615,0.0384615,0.0384615,0.0384615,...,Coffee Shop,Gym,Ice Cream Shop,Market,Bar,Street Food Gathering,German Restaurant,Brazilian Restaurant,Breakfast Spot,Shopping Mall
25,7417 North May,-97.565784,35.545459,"(35.545459, -97.565784)",0.136364,0.0909091,0.0909091,0.0454545,0.0454545,0.0454545,...,Italian Restaurant,Boutique,Bakery,Ice Cream Shop,Automotive Shop,French Restaurant,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Salon / Barbershop


In [42]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 2, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
42,200 Quatum Drive,-97.561006,35.448081,"(35.448081, -97.561006)",0.5,0.5,0,0,0,0,...,Home Service,Hobby Shop,Yoga Studio,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
69,4500 N. Lincoln Blvd,-97.502930,35.516646,"(35.516646, -97.50293)",0.666667,0.333333,0,0,0,0,...,Pizza Place,Museum,French Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
104,"2601 Northwest Expressway, Suite 107 W",-97.572440,35.517614,"(35.517614, -97.57244)",0.5,0.5,0,0,0,0,...,Hookah Bar,Jewelry Store,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
115,308 Northeast 27th Street,-97.506510,35.497600,"(35.4976, -97.50651)",0.5,0.5,0,0,0,0,...,Arts & Crafts Store,Fast Food Restaurant,Yoga Studio,Electronics Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
127,330 Southwest 80th Street,-97.517206,35.386288,"(35.386288, -97.517206)",0.5,0.5,0,0,0,0,...,Sporting Goods Shop,Italian Restaurant,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
145,1240 Northwest 115th Street,-97.534595,35.587175,"(35.587175, -97.534595)",0.5,0.5,0,0,0,0,...,Home Service,Business Service,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
193,3401 S. Meridian Ave.,-97.601124,35.430581,"(35.430581, -97.601124)",0.666667,0.333333,0,0,0,0,...,Rental Car Location,Burger Joint,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [43]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 3, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
6,"8524 South Western, Suite 105",-97.529940,35.381587,"(35.381587, -97.52994)",0.285714,0.285714,0.142857,0.142857,0.142857,0,...,Mexican Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,BBQ Joint,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop
8,"7416 N. Broadway, Suite D",-97.513383,35.546534,"(35.546534, -97.513383)",0.2,0.2,0.2,0.2,0.2,0,...,Photography Studio,Convenience Store,Massage Studio,Golf Course,Gymnastics Gym,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop
20,"3838 NW 36th Street, Suite 108",-97.586443,35.507665,"(35.507665, -97.586443)",0.2,0.2,0.2,0.2,0.2,0,...,Indian Restaurant,Breakfast Spot,Video Store,Bar,Cafeteria,Yoga Studio,Fabric Shop,Food Truck,Food Court,Food & Drink Shop
33,4300 South Shields Boulevard,-97.507773,35.422301,"(35.422301, -97.507773)",0.25,0.25,0.25,0.25,0,0,...,Pharmacy,Fast Food Restaurant,Gas Station,Bowling Alley,Deli / Bodega,Fabric Shop,Food Truck,Cycle Studio,Food Court,Food & Drink Shop
36,"10101 S. Pennsylvania, Suite B",-97.547851,35.366960,"(35.36696, -97.547851)",0.2,0.2,0.2,0.2,0.2,0,...,Pizza Place,Insurance Office,Construction & Landscaping,Gas Station,Mexican Restaurant,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food
38,3100 Southwest 89th Street,-97.571067,35.376960,"(35.37696, -97.571067)",0.2,0.2,0.2,0.2,0.2,0,...,Salon / Barbershop,Sandwich Place,Pharmacy,Gas Station,Liquor Store,Food,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
47,4800 NW 10th Street,-97.607794,35.478626,"(35.478626, -97.607794)",0.25,0.25,0.25,0.25,0,0,...,Auto Workshop,Construction & Landscaping,Home Service,Food,Dry Cleaner,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market
58,"1200 SW 104th Street, Suite D",-97.536274,35.363197,"(35.363197, -97.536274)",0.25,0.25,0.25,0.25,0,0,...,Wings Joint,Home Service,Grocery Store,Athletics & Sports,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food
65,"14000 North Portland, Suite 100",-97.584194,35.614760,"(35.61476, -97.584194)",0.2,0.2,0.2,0.2,0.2,0,...,Massage Studio,Bar,Sandwich Place,Gym / Fitness Center,Tex-Mex Restaurant,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
71,"3501 Northwest 63rd Street, Suite 110-B",-97.579266,35.536833,"(35.536833, -97.579266)",0.2,0.2,0.2,0.2,0.2,0,...,Gas Station,Supermarket,Optical Shop,Rest Area,Gym / Fitness Center,Yoga Studio,Dry Cleaner,Food & Drink Shop,Food,Flower Shop


In [44]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 4, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
24,3301 Northwest 63rd Street,-97.574790,35.536790,"(35.53679, -97.57479)",1,0,0,0,0,0,...,Park,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
68,"1500 NE 4th Street, Suite 105",-97.484775,35.471253,"(35.471253, -97.484775)",1,0,0,0,0,0,...,Deli / Bodega,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
139,3301 Northwest 50th Street,-97.574680,35.522261,"(35.522261, -97.57468)",1,0,0,0,0,0,...,Convenience Store,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
141,1200 Everett Drive,-97.496060,35.480463,"(35.480463, -97.49606)",1,0,0,0,0,0,...,Fast Food Restaurant,Yoga Studio,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service


In [45]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 5, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
4,"4901 West Reno, Suite 950",-97.607725,35.464552,"(35.464552, -97.607725)",0.5,0.25,0.25,0,0,0,...,Hotel,Print Shop,Italian Restaurant,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
19,"9828 Northeast 23rd Street, Suite 2",-97.356617,35.492881,"(35.492881, -97.356617)",0.5,0.25,0.25,0,0,0,...,BBQ Joint,Italian Restaurant,Karaoke Bar,Yoga Studio,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
46,8815 S. Santa Fe Avenue,-97.512514,35.379240,"(35.37924, -97.512514)",0.333333,0.333333,0.333333,0,0,0,...,Antique Shop,Bus Station,Sandwich Place,Yoga Studio,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
102,7200 Broadway Extension,-97.513404,35.544754,"(35.544754, -97.513404)",0.333333,0.333333,0.333333,0,0,0,...,Photography Studio,Massage Studio,Gymnastics Gym,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
109,"701 Northeast 10th Street, Suite 124",-97.500703,35.478444,"(35.478444, -97.500703)",0.333333,0.333333,0.333333,0,0,0,...,Bookstore,Medical Center,Gym,Furniture / Home Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
142,940 Northeast 13th Street,-97.496303,35.482153,"(35.482153, -97.496303)",0.333333,0.333333,0.333333,0,0,0,...,Playground,Fast Food Restaurant,Gym,Yoga Studio,Dry Cleaner,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
149,"701 NW 63rd, Suite 102",-97.522617,35.536961,"(35.536961, -97.522617)",0.333333,0.333333,0.333333,0,0,0,...,Cosmetics Shop,Seafood Restaurant,Restaurant,Donut Shop,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
166,3524 Northwest 56th Street,-97.579714,35.527659,"(35.527659, -97.579714)",0.5,0.25,0.25,0,0,0,...,Pharmacy,Gift Shop,Bar,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
183,"13313 North Meridian Avenue, Building B",-97.603032,35.606128,"(35.606128, -97.603032)",0.333333,0.333333,0.333333,0,0,0,...,Mediterranean Restaurant,Medical Center,Gift Shop,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [46]:
oklahoma_merged.loc[oklahoma_merged['Cluster Labels'] == 6, oklahoma_merged.columns[[1] + list(range(5, oklahoma_merged.shape[1]))]]

,ADDRESS,LONGITUDE,LATITUDE,LOCATION,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,1st Most Common Venue_name,2nd Most Common Venue_name,3rd Most Common Venue_name,4th Most Common Venue_name,5th Most Common Venue_name,6th Most Common Venue_name,7th Most Common Venue_name,8th Most Common Venue_name,9th Most Common Venue_name,10th Most Common Venue_name
2,"4350 Will Rogers Parkway, Suite 500",-97.597747,35.452096,"(35.452096, -97.597747)",0.176471,0.117647,0.117647,0.0588235,0.0588235,0.0588235,...,Mexican Restaurant,Hotel,Burger Joint,Steakhouse,BBQ Joint,Breakfast Spot,Business Service,Liquor Store,Arts & Crafts Store,Dive Bar
11,"4801 N. Classen, Suite 237",-97.534535,35.519655,"(35.519655, -97.534535)",0.222222,0.166667,0.111111,0.111111,0.0555556,0.0555556,...,Hotel,Breakfast Spot,Mexican Restaurant,Dive Bar,Gas Station,Bar,Optical Shop,Coffee Shop,Intersection,Pizza Place
14,9700 Mashburn Boulevard,-97.654939,35.568751,"(35.568751, -97.654939)",0.25,0.166667,0.0833333,0.0833333,0.0833333,0.0833333,...,Fast Food Restaurant,Pharmacy,Italian Restaurant,Pizza Place,Dessert Shop,Gas Station,Thrift / Vintage Store,Video Store,Convenience Store,Health & Beauty Service
21,3105 S. Meridian,-97.601164,35.433842,"(35.433842, -97.601164)",0.2,0.2,0.1,0.1,0.1,0.1,...,Pizza Place,Rental Car Location,Bowling Alley,American Restaurant,Outdoor Supply Store,Construction & Landscaping,Burger Joint,Gas Station,Flea Market,Fast Food Restaurant
27,"One N. W. 64th Street, Suite A",-97.512180,35.537877,"(35.537877, -97.51218)",0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,...,Flower Shop,Gym / Fitness Center,Hotel,Home Service,Medical Center,Gas Station,Electronics Store,Food Court,Food & Drink Shop,Food
29,"5401 North Portland, Suite 280",-97.583505,35.525317,"(35.525317, -97.583505)",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,...,Breakfast Spot,Smoke Shop,Gas Station,Bar,Bakery,Grocery Store,Sandwich Place,Food Truck,Food Court,Food & Drink Shop
31,3909 South Western Avenue,-97.530336,35.425752,"(35.425752, -97.530336)",0.222222,0.111111,0.111111,0.111111,0.111111,0.111111,...,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,Pizza Place,American Restaurant,Department Store,Discount Store,Grocery Store,Food Court,Food & Drink Shop
32,3714 N. Portland,-97.583222,35.508067,"(35.508067, -97.583222)",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,...,Indian Restaurant,Breakfast Spot,Video Store,Bar,Grocery Store,Fast Food Restaurant,Cafeteria,Yoga Studio,Fabric Shop,Food Court
45,3824 North Meridian #101,-97.601034,35.509334,"(35.509334, -97.601034)",0.222222,0.111111,0.111111,0.111111,0.111111,0.111111,...,Bar,BBQ Joint,Mexican Restaurant,Gift Shop,Video Store,Residential Building (Apartment / Condo),Donut Shop,Fast Food Restaurant,Yoga Studio,Food Court
49,4201 South Western,-97.530363,35.422301,"(35.422301, -97.530363)",0.2,0.1,0.1,0.1,0.1,0.1,...,Fast Food Restaurant,Pharmacy,Department Store,Mobile Phone Shop,Sandwich Place,Grocery Store,Discount Store,Mexican Restaurant,Pizza Place,Design Studio
